In [49]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import TextVectorization
from numpy import array

In [50]:
XOR = [ (array([0,0]), 0),
        (array([0,1]), 1),
        (array([1,0]), 1),
        (array([1,1]), 0), ]

X_train = array([x[0] for x in XOR])
y_train = array([x[1] for x in XOR])

In [51]:
network = Sequential()
network.add(Input((2,)))
network.add(Dense(2, activation='relu'))
network.add(Dense(1, activation='sigmoid'))
network.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

In [52]:
network.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

In [53]:
network.fit(x=X_train,y=y_train)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.7500 - loss: 0.7669


In [54]:
int((network.predict(array([[0, 1]])) > 0.5)[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step


1

In [55]:
int((network.predict(array([[0, 0]])) > 0.5)[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


0

### **ELABORATION DE NOTRE BASE D'APPRENTISSAGE DES LANGUES**

In [56]:
import pandas as pd

# Charger les fichiers de texte anglais et français
with open('english.txt', 'r') as file:
    english_text = file.read().strip()

with open('french.txt', 'r') as file:
    french_text = file.read().strip()
# Nettoyer le texte en retirant les sauts de ligne
english_text_nettoye = english_text.replace('\n', '')
french_text_nettoye = french_text.replace('\n', '')
# Séparer les textes en phrases
english_phrases = english_text_nettoye.split('.')
french_phrases = french_text_nettoye.split('.')
# Nous allons garder que les phrases qui contiennent plus de 5 mots et eliminer les phrases en anglais(Les credits en fin de livre) présents dans les textes
english_phrases_filtrees = [phrase for phrase in english_phrases if len(phrase.split()) > 5]
french_phrases_filtrees = [phrase for phrase in french_phrases[:-250] if len(phrase.split()) > 5]
# Créer un dataframe avec les phrases anglaises et françaises filtrées
df = pd.DataFrame({'texte': english_phrases_filtrees + french_phrases_filtrees, 'langue': ['anglais']*len(english_phrases_filtrees) + ['français']*len(french_phrases_filtrees)})
df

,texte,langue
0,﻿The Project Gutenberg eBook of The Complete W...,anglais
1,"You may copy it, give it away or re-use it un...",anglais
2,"If you are not located in the United States,y...",anglais
3,Title: The Complete Works of William Shakespea...,anglais
4,2When forty winters shall ...,anglais
...,...,...
51554,Javert aperçut la soeur et s'arrêta interdit,français
51555,"On se rappelle que le fond même de Javert, son...",français
51556,Il était tout d'unepièce et n'admettait ni ob...,français
51557,"Pour lui, bienentendu, l'autorité ecclésiasti...",français


#### **PREPARATION DES DONNÉES À L'ENTRAINEMENT**

In [57]:
vectorizer = TextVectorization(max_tokens=10000, output_mode='tf-idf')
vectorizer.adapt(df['texte'])
# Vectoriser les données
X = vectorizer(df['texte'])
# Encoder les labels 
y = tf.where(df['langue'] == 'français', 1, 0)
# Diviser les données en ensemble d'entraînement et de test
TAILLE_TEST = int(0.2 * len(X))
indices = tf.range(len(X))
indices_melanges = tf.random.shuffle(indices)
X = tf.gather(X, indices_melanges)
y = tf.gather(y, indices_melanges)
X_train = X[TAILLE_TEST:]
X_test = X[:TAILLE_TEST]
y_train = y[TAILLE_TEST:]
y_test = y[:TAILLE_TEST]


#### **CREATION DU MODÈLE**

In [58]:
# le model 
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Entrainement 
model.fit(X_train, y_train, epochs=10, batch_size=10)
# Évaluation du modèle
_, accuracy = model.evaluate(X_test, y_test)
print('Précision: %.2f' % (accuracy*100))


/Users/Afdal/Documents/GitHub/Neural-networks-class-exercises/Neural-networks-class-exercises/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9869 - loss: 0.0599
Epoch 2/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - accuracy: 0.9994 - loss: 0.0028
Epoch 3/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9997 - loss: 0.0011
Epoch 4/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9999 - loss: 5.3881e-04
Epoch 5/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9999 - loss: 4.4550e-04
Epoch 6/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9998 - loss: 4.2405e-04
Epoch 7/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9999 - loss: 3.6835e-04
Epoch 8/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9998 - loss: 4.3849e-04
Epoch 9/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9998 - loss: 3.2930e-04
Epoch 10/10
4125/4125 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.9999 - loss: 3.3230e-04
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9997 - loss: 0.0044
Précis

#### **TEST**

In [59]:
# Tester le modèle avec une nouvelle phrase
phrase = "Je me nomme Afdal"
inputs = vectorizer([phrase])
prediction = model(inputs, training=False)
prediction_binaire = tf.where(prediction >= 0.5, 1, 0)
print("Prédiction:", prediction_binaire.numpy()[0][0])

Prédiction: 1


In [60]:
phrase = "My name is Afdal"
inputs = vectorizer([phrase])
prediction = model(inputs, training=False)
prediction_binaire = tf.where(prediction >= 0.5, 1, 0)
print("Prédiction:", prediction_binaire.numpy()[0][0])

Prédiction: 0


#### **DANS LE CAS DE PLUSIEURS LANGUES (4)**

In [61]:
import pandas as pd

# Charger les fichiers de texte pour chaque langue
with open('english.txt', 'r') as file:
    english_text = file.read().strip()

with open('french.txt', 'r') as file:
    french_text = file.read().strip()

with open('espagnol.txt', 'r') as file:
    espagnol_text = file.read().strip()

with open('finnish.txt', 'r') as file:
    finnish_text = file.read().strip()

# Nettoyer le texte en retirant les sauts de ligne \n
english_text_nettoye = english_text.replace('\n', '')
french_text_nettoye = french_text.replace('\n', '')
espagnol_text_nettoye = espagnol_text.replace('\n', '')
finnish_text_nettoye = finnish_text.replace('\n', '')

#  On sépare les textes en phrase
english_phrases = english_text_nettoye.split('.')
french_phrases = french_text_nettoye.split('.')
espagnol_phrases = espagnol_text_nettoye.split('.')
finnish_phrases = finnish_text_nettoye.split('.')

# Nous allons garder que les phrases qui contiennent plus de 5 mots et eliminer les phrases en anglais(Les credits en fin de livre) présents dans les textes
english_phrases_filtrees = [phrase for phrase in english_phrases if len(phrase.split()) > 5]
french_phrases_filtrees = [phrase for phrase in french_phrases[:-250] if len(phrase.split()) > 5]
espagnol_phrases_filtrees = [phrase for phrase in espagnol_phrases[:-250] if len(phrase.split()) > 5]
finnish_phrases_filtrees = [phrase for phrase in finnish_phrases[:-250] if len(phrase.split()) > 5]

df = pd.DataFrame({
    'texte': english_phrases_filtrees + french_phrases_filtrees + espagnol_phrases_filtrees + finnish_phrases_filtrees,
    'langue': ['anglais']*len(english_phrases_filtrees) + ['français']*len(french_phrases_filtrees) + ['espagnol']*len(espagnol_phrases_filtrees) + ['finnois']*len(finnish_phrases_filtrees)
})
df

,texte,langue
0,﻿The Project Gutenberg eBook of The Complete W...,anglais
1,"You may copy it, give it away or re-use it un...",anglais
2,"If you are not located in the United States,y...",anglais
3,Title: The Complete Works of William Shakespea...,anglais
4,2When forty winters shall ...,anglais
...,...,...
52932,Hän peitti kettu-naisenkuin pienen lapsen huo...,finnois
52933,Yöstä huolimatta jatkoi matkue kulkuansa Tokioon,finnois
52934,Iltaruoka oliviety koskemattomana takaisin; p...,finnois
52935,Hän katsoa tuijotti jotakin kädessäänolevaa e...,finnois


#### **PREPARATION DES DONNÉES À L'ENTRAINEMENT**

In [62]:

vectorizer = TextVectorization(max_tokens=10000, output_mode='tf-idf')
vectorizer.adapt(df['texte'])

# Vectoriser les données
X = vectorizer(df['texte'])

# Encoder les labels 
y = tf.where(df['langue'] == 'français', 0, 
    tf.where(df['langue'] == 'anglais', 1, 
    tf.where(df['langue'] == 'espagnol', 2, 3)))

# Diviser les données en ensemble d'entraînement et de test
TAILLE_TEST = int(0.2 * len(X))
indices = tf.range(len(X))
indices_melanges = tf.random.shuffle(indices)

X = tf.gather(X, indices_melanges)
y = tf.gather(y, indices_melanges)

X_train = X[TAILLE_TEST:]
X_test = X[:TAILLE_TEST]
y_train = y[TAILLE_TEST:]
y_test = y[:TAILLE_TEST]


#### **DEFINITION DU MODÈLE**

In [63]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(X.shape[1],)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


/Users/Afdal/Documents/GitHub/Neural-networks-class-exercises/Neural-networks-class-exercises/.venv/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 128)            │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,644 (4.89 MB)

 Trainable params: 1,280,644 (4.89 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 28s 21ms/step - accuracy: 0.9824 - loss: 0.0824 - val_accuracy: 0.9983 - val_loss: 0.0100
Epoch 2/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - accuracy: 0.9994 - loss: 0.0022 - val_accuracy: 0.9982 - val_loss: 0.0070
Epoch 3/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - accuracy: 0.9997 - loss: 0.0011 - val_accuracy: 0.9984 - val_loss: 0.0105
Epoch 4/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 32s 24ms/step - accuracy: 0.9997 - loss: 9.3779e-04 - val_accuracy: 0.9982 - val_loss: 0.0114
Epoch 5/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 39s 29ms/step - accuracy: 0.9996 - loss: 0.0019 - val_accuracy: 0.9984 - val_loss: 0.0122
Epoch 6/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 40s 30ms/step - accuracy: 0.9998 - loss: 8.6816e-04 - val_accuracy: 0.9981 - val_loss: 0.0150
Epoch 7/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - accuracy: 0.9998 - loss: 7.8753e-04 - val_accuracy: 0.9984 - val_loss: 0.0160
Epoch 8/10
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - accura

## **TEST**

In [64]:
exemples_phrases = [
    "Rester soi-même dans un monde qui tente constamment de te changer est le plus grand accomplissement.",
    "Retiens ceci mon élève : If you're determined to learn, no one can stop you.",
    "Más vale prevenir que curar",
    "Hotellivirkailija: Huoneissa tai parvekkeilla ei saa tupakoida.Teemu: Tupakoiminen on vaarallista terveydelle.Hotellivirkailija: Aivan"
]

X_exemples = vectorizer(exemples_phrases)

# Nous allons prédire les langues pour les exemples de texte
y_pred_exemples = model.predict(X_exemples)
y_pred_exemples_classes = tf.argmax(y_pred_exemples, axis=1)


for i, phrase in enumerate(exemples_phrases):
    print(f"Phrase: {phrase}")
    print(f"Langue prédite: {['français', 'anglais', 'espagnol', 'finnois'][y_pred_exemples_classes[i]]}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Phrase: Rester soi-même dans un monde qui tente constamment de te changer est le plus grand accomplissement.
Langue prédite: français

Phrase: Retiens ceci mon élève : If you're determined to learn, no one can stop you.
Langue prédite: anglais

Phrase: Más vale prevenir que curar
Langue prédite: espagnol

Phrase: Hotellivirkailija: Huoneissa tai parvekkeilla ei saa tupakoida.Teemu: Tupakoiminen on vaarallista terveydelle.Hotellivirkailija: Aivan
Langue prédite: finnois

